# Chapter 17: Custom Training with TensorFlow

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 17 - Custom Training with TensorFlow** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini menjelaskan cara melakukan proses training secara manual (low-level) menggunakan GradientTape, loss custom, loop training kustom, dan kontrol penuh terhadap update parameter.

---


## Ringkasan Teori Bab 17: Custom Training with TensorFlow

### 1. Kapan Butuh Custom Training?

- Saat kita butuh **kontrol penuh** atas proses training
- Misalnya untuk:
  - Logging dan metrik kustom
  - Update parameter secara berbeda per layer
  - Perhitungan loss khusus

---

### 2. GradientTape

`tf.GradientTape()` digunakan untuk merekam operasi agar kita bisa menghitung gradien secara eksplisit.

```python
with tf.GradientTape() as tape:
    y_pred = model(x_batch)
    loss = loss_fn(y_batch, y_pred)
grads = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))
```

---

### 3. Membuat Loop Training Manual

Langkah utama dalam training loop:
1. Ambil batch dari dataset
2. Hitung prediksi dan loss
3. Hitung gradien dengan `GradientTape`
4. Update bobot menggunakan `optimizer`

---

### 4. Custom Loss Function

Kita bisa membuat fungsi loss sendiri:
```python
def custom_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))
```

Bisa dipakai langsung saat compile model:
```python
model.compile(loss=custom_mse, ...)
```

---

### 5. Training Step Kustom (dalam Class Model)

Dengan subclassing `tf.keras.Model`, kita bisa override:
- `train_step(self, data)`
- `test_step(self, data)`
- `compile(...)` tetap digunakan

---

### 6. Logging dan Metric

Gunakan:
- `tf.keras.metrics.Mean()` untuk menyimpan nilai loss rata-rata
- `reset_states()` dipanggil tiap epoch
- `update_state()` dipanggil di setiap batch

---

### 7. Checkpoint & Callback

- Checkpoint manual dapat dibuat dengan `tf.train.Checkpoint`
- Gunakan `tf.keras.callbacks.Callback` untuk logging, early stopping, dsb.

---

📌 Kesimpulan:
Bab ini memberikan kita kekuatan untuk mengontrol proses training dari nol, cocok untuk riset atau eksperimen model yang tidak konvensional.

---


## Implementasi Custom Training Loop

Contoh ini akan menunjukkan bagaimana melakukan proses training secara manual dengan:
- Subclassing `Model`
- `GradientTape`
- Logging manual metrik akurasi & loss

Dataset: Iris (klasifikasi 3 kelas)

---


In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load dan siapkan data
iris = load_iris()
X = iris.data
y = to_categorical(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Subclassing model
class CustomIrisModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(10, activation='relu')
        self.output_layer = tf.keras.layers.Dense(3, activation='softmax')

    def call(self, inputs):
        x = self.hidden(inputs)
        return self.output_layer(x)

model = CustomIrisModel()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


In [ ]:
# Loop training manual
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = loss_fn(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss.update_state(loss)
    train_accuracy.update_state(y, predictions)

@tf.function
def test_step(x, y):
    predictions = model(x)
    t_loss = loss_fn(y, predictions)
    test_loss.update_state(t_loss)
    test_accuracy.update_state(y, predictions)

# Training loop
EPOCHS = 30
for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for x_batch, y_batch in train_dataset:
        train_step(x_batch, y_batch)

    for x_batch, y_batch in test_dataset:
        test_step(x_batch, y_batch)

    print(f"Epoch {epoch+1}: "
          f"Train Loss={train_loss.result():.4f}, Train Acc={train_accuracy.result():.4f}, "
          f"Test Loss={test_loss.result():.4f}, Test Acc={test_accuracy.result():.4f}")


### Penjelasan:
- `CustomIrisModel` adalah model kustom berbasis subclassing
- Loop training ditulis secara manual dengan `GradientTape`
- Kita update metrik dan optimizer secara eksplisit
- Pendekatan ini cocok untuk riset, eksperimen loss khusus, atau arsitektur tak biasa

📌 Perlu kontrol penuh? Gunakan pendekatan ini.

---
